In [2]:
import glob
import itertools
import math
import os
import sys
import re

import numpy as np
import pandas as pd
import scipy

import spacy
import textstat
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction
from nltk.corpus import stopwords

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [3]:
gloveFile = "glove.6B.50d.txt"

# Define Metrics

In [4]:
originalPath = "/home/ubuntu/style-transformer/outputs/soph_1/model_iteration_lr_0.0001/"
higherLR = "/home/ubuntu/style-transformer/outputs/soph_1/model_iteration_lr_0.001/"
soph2 = "/home/ubuntu/style-transformer/outputs/soph_2/"
soph3 = "/home/ubuntu/style-transformer/outputs/soph_3/"
sophTagged = "/home/ubuntu/style-transformer/outputs/soph_tagged/"
sophTaggedNp = "/home/ubuntu/style-transformer/outputs/soph_tagged_np/"


In [5]:
def process(sent):
    sent = sent.strip().replace('<pad>', '').strip()
    return sent

In [6]:
def readNaiveTest(runNum):
    path = f"/home/ubuntu/style-transformer/data/soph_{runNum}/test.neg"
    with open(path) as f:
        naive = f.readlines()
    return list(map(process, naive))

In [7]:
def load_transformer(path):
    with open(path + "gold_text.txt") as f:
        gold = f.readlines()
    with open(path + "rev_output_0.txt") as f:
        rev0 = f.readlines()
    with open(path + "raw_output_0.txt") as f:
        raw0 = f.readlines()
    with open(path + "rev_output_1.txt") as f:
        rev1 = f.readlines()
    with open(path + "raw_output_1.txt") as f:
        raw1 = f.readlines()
        
    gold = list(map(process, gold))
    rev0 = list(map(process, rev0))
    raw0 = list(map(process, raw0))
    
    return {0: (gold, rev0, raw0), 1:(gold, rev1, raw1)}

### BLEU

In [8]:
def bleu_sent(originText, transferredText):
    texts_origin = [
        word_tokenize(text.lower().strip()) 
        for text in originText
    ]
    text_transfered = word_tokenize(transferredText.lower().strip())
    cc = SmoothingFunction()
    return sentence_bleu(texts_origin, text_transfered, smoothing_function=cc.method3)



def bleu_avg(originText, transferredText):
    sum = 0
    n = len(originText)
    for x, y in zip(originText, transferredText):
        sum += bleu_sent([x], y)
    return sum / n

### KenLM

LMs add probability to each token sequence to indicate how likely it is for the sequence to occur in real text. Train LM on the target language, and the model estimates the probability of seeing a given sentence in the target text using Markov chains.


In information theory, perplexity is a measurement of how well a probability distribution or probability model predicts a sample. It may be used to compare probability models. A low perplexity indicates the probability distribution is good at predicting the sample. 

The perplexity(sometimes called PP for short) of a language model on a test set is the inverse probability of the test set, normalized by the numberof words. https://lagunita.stanford.edu/c4x/Engineering/CS-224N/asset/slp4.pdf

PPLxdenotes theperplexity of sentences transferred from positive sentences evaluated by a language model trainedwith negative sentences and vice versa. https://arxiv.org/pdf/1805.11749.pdf

In [9]:
def load_kenlm():
    global kenlm
    import kenlm

def train_ngram_lm(kenlm_path, data_path, output_path, N, load=False):
    if not load:
        
        curdir = os.path.abspath(os.path.curdir)
        command = "bin/lmplz -o "+str(N)+" <"+os.path.join(curdir, data_path) + \
                  " >"+os.path.join(curdir, output_path)
        print(command)
        os.system("cd "+os.path.join(kenlm_path, 'build')+" && "+command)

    load_kenlm()

    assert(output_path) 
    model = kenlm.Model(output_path)

    return model

In [ ]:
def SentencePplFrame(reference, transferred, klm):
    ppl_dict = {}
    for i in range(len(reference)):
        ppl_dict[i] = {'ppl':(get_ppl(klm, [reference[i]]), get_ppl(klm, [transferred[i]])), 
                       'sent1': reference[i],
                       'sent2': transferred[i]}
    test_df = pd.DataFrame(ppl_dict).T
    test_df['ppl1'] = test_df.ppl.apply(lambda x: x[0])
    test_df['ppl2'] = test_df.ppl.apply(lambda x: x[1])
    test_df = test_df.sort_values('ppl2')
    cols = ['ppl1', 'ppl2', 'sent1', 'sent2']
    
    return test_df[cols]

In [9]:
kenlm_model = train_ngram_lm(
    'kenlm', 
    'data/processed/soph_train_tagged_nopunct.txt', 
    'klm_soph_tagged_np.arpa', 
    5, 
    load=False
)

bin/lmplz -o 5 </home/ubuntu/CS_230_Project/data/processed/soph_train_tagged_nopunct.txt >/home/ubuntu/CS_230_Project/klm_soph_tagged_np.arpa


In [46]:
sentence = gold[10]
# Show scores and n-gram matches
words = ['<s>'] + sentence.split() + ['</s>']
for i, (prob, length, oov) in enumerate(kenlm_model.full_scores(sentence)):
    print('{0} {1}: {2}'.format(prob, length, ' '.join(words[i+2-length:i+2])))
    if oov:
        print('\t"{0}" is an OOV'.format(words[i+1]))

# Find out-of-vocabulary words
for w in words:
    if not w in kenlm_model:
        print('"{0}" is an OOV'.format(w))

-1.716026782989502 2: <s> they
-4.684571266174316 1: follow
-3.069218635559082 1: what
-5.896757125854492 1: technology
	"technology" is an OOV
-1.8870635032653809 1: to
-1.8208752870559692 2: to make
-3.4363248348236084 2: make life
-4.816084384918213 1: .
-0.05439453572034836 2: . </s>
"technology" is an OOV


In [10]:
def get_ppl(lm, sentences):
    """
    Assume sentences is a list of strings (space delimited sentences)
    """
    total_nll = 0
    total_wc = 0
    for sent in sentences:
        words = sent.strip().split()
        nll = np.sum([- math.log(math.pow(10.0, score)) for score, _, _ in lm.full_scores(sent, bos=True, eos=False)])
        word_count = len(words)
        total_wc += word_count
        total_nll += nll
    ppl = np.exp(total_nll / total_wc)
    return ppl

### Similarities - Jaccard,  Cosine

In [11]:
def jaccard_sim(sent1, sent2): 
    a = set(sent1.split()) 
    b = set(sent2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [12]:
def loadGloveModel(gloveFile):
    with open(gloveFile, encoding="utf8" ) as f:
        content = f.readlines()
    model = {}
    for line in content:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    return model

def cosine_format(raw):

    processed = re.sub("[^a-zA-Z]", " ", raw)
    words = processed.lower().split()
    stopword_set = set(stopwords.words("english"))
    uniq_words = list(set([w for w in words if w not in stopword_set]))
    
    return uniq_words

def cosine_words(word1, word2):

    return (1 - scipy.spatial.distance.cosine(model[word1], model[word2]))


In [13]:
model = loadGloveModel(gloveFile)

In [14]:
def cosine_sent(sent1, sent2):
    if not isinstance(sent1, list):
        sent1 = cosine_format(sent1)
        sent2 = cosine_format(sent2)

    embs1 = np.mean([model[word] for word in sent1], axis=0)
    embs2 = np.mean([model[word] for word in sent2], axis=0)

    return(1 - scipy.spatial.distance.cosine(embs1, embs2))

In [14]:

def heat_matrix(sent1, sent2):
    
    s1 = cosine_format(sent1)
    s2 = cosine_format(sent2)
    
    result_list = [[cosine_words(word1, word2) for word2 in s2] for word1 in s1]
    result_df = pd.DataFrame(result_list)
    result_df.columns = s2
    result_df.index = s1
    
    return result_df


def heat_map(s1, s2):
    df = heat_matrix(s1, s2)

    fig, ax = plt.subplots(figsize=(5,5)) 
    ax_blue = sns.heatmap(df, cmap="YlGnBu")
    print(cosine_sent(s1, s2))
    return ax_blue

### PINC 

https://github.com/cocoxu/Shakespeare/blob/master/python/PINC_sentence.py

In [30]:
def intersect(list1, list2) :
    cnt1 = Counter()
    cnt2 = Counter()
    for tk1 in list1:
        cnt1[tk1] += 1
    for tk2 in list2:
        cnt2[tk2] += 1    
    inter = cnt1 & cnt2
    return len(list(inter.elements()))


def pinc(ssent, csent):
    s1grams = ssent.split(" ")
    c1grams = csent.split(" ")
    s2grams = []
    c2grams = []
    s3grams = []
    c3grams = []
    s4grams = []
    c4grams = []
        
    for i in range(0, len(s1grams)-1) :
        if i < len(s1grams) - 1:
            s2gram = s1grams[i] + " " + s1grams[i+1]
            s2grams.append(s2gram)
        if i < len(s1grams)-2:
            s3gram = s1grams[i] + " " + s1grams[i+1] + " " + s1grams[i+2]
            s3grams.append(s3gram)
        if i < len(s1grams)-3:
            s4gram = s1grams[i] + " " + s1grams[i+1] + " " + s1grams[i+2] + " " + s1grams[i+3]
            s4grams.append(s4gram)
            
    for i in range(0, len(c1grams)-1) :
        if i < len(c1grams) - 1:
            c2gram = c1grams[i] + " " + c1grams[i+1]
            c2grams.append(c2gram)
        if i < len(c1grams)-2:
            c3gram = c1grams[i] + " " + c1grams[i+1] + " " + c1grams[i+2]
            c3grams.append(c3gram)
        if i < len(c1grams)-3:
            c4gram = c1grams[i] + " " + c1grams[i+1] + " " + c1grams[i+2] + " " + c1grams[i+3]
            c4grams.append(c4gram)

    score = intersect(s1grams, c1grams) / len(c1grams)
    if len(c2grams) > 0:
        score += intersect(s2grams, c2grams) / len(c2grams)
    if len(c3grams) > 0:
        score += intersect(s3grams, c3grams) / len(c3grams)
    if len(c4grams) > 0:
        score += intersect(s4grams, c4grams) / len(c4grams)
    return 1 - score/4


In [31]:
def pinc_corpus(origText, transferText):
    sentcount = len(origText)
    pincscore = 0.0

    for idx in range(len(origText)):

        sline = origText[idx].strip()
        cline = transferText[idx].strip()
        
        sentscore = pinc(sline, cline)       
        pincscore += sentscore

    pincscore = pincscore / sentcount * 100
    
    return pincscore

### Putting it all together

In [32]:
def sentenceMetrics(sent1, sent2, kenlm_model, output=False):
    metrics = {}
    
    metrics['bleu'] = bleu_sent(sent1, sent2)
    metrics['cosine'] = cosine_sent(sent1, sent2)
    metrics['jaccard'] = jaccard_sim(sent1, sent2)
    metrics['pinc'] = pinc(sent1, sent2)
    metrics['ppl'] = (get_ppl(kenlm_model, [sent1]), get_ppl(kenlm_model, [sent2]))
    
    if output:
        print(f"Orig: {sent1}")
        print(f"New: {sent2}")
        heat_map(sent1, sent2)
    return metrics

In [19]:
def globalMetrics(origData, transferData, kenlm_model):
    metrics = {}
    
    metrics['bleu'] = bleu_avg(origData, transferData)
    metrics['ppl'] = (get_ppl(kenlm_model, origData), 
                      get_ppl(kenlm_model, transferData))
    metrics['pinc'] = pinc_corpus(origData, transferData)
    
    return metrics

### Dataset Metrics

In [19]:
loaded_data = load_transformer(originalPath)
gold_orig, rev_orig, raw_orig = loaded_data[0]

In [20]:
loaded_data = load_transformer(higherLR)
gold_HLR, rev_HLR, raw_HLR = loaded_data[0]

In [21]:
loaded_data = load_transformer(soph2)
gold_soph2, rev_soph2, raw_soph2 = loaded_data[0]

In [22]:
loaded_data = load_transformer(sophTagged)
gold_soph_tag, rev_soph_tag, raw_soph_tag = loaded_data[0]

In [23]:
loaded_data = load_transformer(sophTaggedNp)
gold_soph_tag_np, rev_soph_tag_np, raw_soph_tag_np = loaded_data[0]

In [24]:
naive_1 = readNaiveTest(1)
naive_2 = readNaiveTest(2)
naive_3 = readNaiveTest(3)
naive_tag = readNaiveTest('tagged')
naive_tag_np = readNaiveTest('tagged_np')

In [25]:
# kenlm_1 = train_ngram_lm('kenlm', 'data/processed/soph_train.txt', 'klm_soph_1.arpa', 5, load=True)
# kenlm_2 = train_ngram_lm('kenlm', 'data/processed/soph_train_2.txt', 'klm_soph_2.arpa', 5, load=True)
# kenlm_3 = train_ngram_lm('kenlm', 'data/processed/soph_train_3.txt', 'klm_soph_3.arpa', 5, load=True)
# kenlm_tag = train_ngram_lm('kenlm', 'data/processed/soph_train_tagged.txt', 'klm_soph_tagged.arpa', 5, load=True)
kenlm_tag_np = train_ngram_lm('kenlm', 'data/processed/soph_train_tagged_nopunct.txt', 'klm_soph_tagged_np.arpa', 5, load=True)

In [105]:
globalMetrics(naive_2, rev_soph2, kenlm_2)

{'bleu': 0.3613066422495174,
 'ppl': (798.1598083846517, 629.9162131290562),
 'pinc': 62.63539704549997}

In [26]:
globalMetrics(naive_tag, rev_soph_tag, kenlm_tag)

{'bleu': 0.380887868597427,
 'ppl': (447.7485862016299, 282.4529932116936),
 'pinc': 58.73869506288379}

In [31]:
globalMetrics(naive_tag_np, rev_soph_tag_np, kenlm_tag_np)

{'bleu': 0.09690859889981704,
 'ppl': (546.7043494507218, 73.09098323777687),
 'pinc': 83.82863839728782}

In [33]:
dftag = SentencePplFrame(naive_tag_np, rev_soph_tag_np, kenlm_tag_np)

In [34]:
pd.set_option('display.max_colwidth', -1)
dftag.sample(frac=1).head(100)

ppl1        ppl2  \
995   482.787222   54.211250    
318   1257.029526  45.988393    
443   980.760771   101.954429   
803   423.573330   69.036768    
525   330.456966   67.816189    
1412  827.701177   62.681532    
1575  907.726414   49.519487    
435   2061.942170  157.430043   
572   1226.238892  74.116477    
1581  2550.839142  635.986420   
642   1667.006319  574.505827   
1673  360.025208   60.302671    
1150  542.009250   39.261224    
1256  723.978158   72.689014    
640   499.350265   45.673305    
399   570.628212   109.817339   
1113  948.893445   48.551498    
115   638.058207   30.625649    
1174  485.969761   139.408003   
1483  98.255765    83.834317    
1148  1399.538719  163.137864   
820   541.468522   103.616829   
1356  299.209189   31.721332    
207   1564.997506  82.377836    
861   116.446454   50.639326    
250   49.131490    24.010395    
446   1313.372109  118.414855   
1540  257.144343   87.898236    
1027  1325.727417  58.239437    
865   341.425682   80.445854    
...          ...         ...    
32    383.719745   63.652561    
1182  305.784700   54.745153    
958   148.163990   108.185307   
600   1737.729518  55.314201    
1052  637.240881   60.786237    
234   625.256345   42.246590    
757   130.568066   78.129123    
188   211.132560   50.629581    
1635  293.973794   16.317459    
526   182.816458   67.317436    
1033  539.108789   138.852144   
1582  4044.322503  185.894541   
1612  1633.287639  86.445822    
257   181.789263   64.810319    
1072  105.479998   74.750426    
312   288.736308   138.395285   
1473  190.535639   50.716945    
936   1061.461716  68.017508    
1566  326.781090   47.604413    
1527  227.011267   68.025145    
681   2054.252542  131.725294   
1074  337.288633   152.305297   
986   1111.514086  47.518364    
953   1732.553963  88.069953    
776   1592.399370  176.473670   
1345  517.747634   188.400909   
1318  584.096375   75.798493    
1440  279.189413   78.757911    
951   503.292554   150.884318   
758   1076.980960  146.520470   

                                                                                                                                                           sent1  \
995   so everyday i had to try to keep my mind on something besides the phone                                                                                      
318   the feature of rolling hills also affects the cyclist                                                                                                        
443   the hibiscus does these actions by coming out of its buds and blooming                                                                                       
803   when i got home i had learn two lessons one was not to worry about anything and i learn how to have patients                                                 
525   i know that after a long time on the computer i have a massive headache because my eyes just hurt so bad                                                     
1412  nature itself also seems to <caps> a <caps> the winds above the building were very violent                                                                   
1575  in the story do not exceed posted speed limit the cyclist sees many different settings when he left it was early summer and it was pretty hot outside        
435   in conclusion you live a happier life when explore the                                                                                                       
572   the human race has survived over hundreds of years without computers and we can survive hundreds more                                                        
1581  libraries are usually pretty good at sorting things                                                                                                          
642   the other practical reason why dirigibles could not more at the empire state building was an existing law against airships flying too low over urban areas  

In [28]:
#https://en.wikipedia.org/wiki/Flesch%E2%80%93Kincaid_readability_tests

list(map(textstat.flesch_kincaid_grade, rev_soph_tag_np))

[-1.5,
 5.2,
 2.8,
 -2.7,
 1.3,
 7.2,
 0.5,
 5.6,
 2.4,
 1.3,
 2.1,
 4.4,
 -1.5,
 6.0,
 4.8,
 3.2,
 3.6,
 4.8,
 1.3,
 1.3,
 3.6,
 1.3,
 1.3,
 0.1,
 5.2,
 -1.2,
 -0.8,
 1.7,
 5.6,
 -0.4,
 1.3,
 5.2,
 3.6,
 2.1,
 0.9,
 2.8,
 5.2,
 0.9,
 4.0,
 2.9,
 5.6,
 7.1,
 6.4,
 3.3,
 4.8,
 8.0,
 4.8,
 -0.4,
 0.5,
 -1.5,
 1.3,
 5.2,
 -1.5,
 -0.4,
 3.3,
 0.5,
 -0.4,
 2.8,
 3.2,
 10.6,
 -1.9,
 4.8,
 4.4,
 0.5,
 0.5,
 1.7,
 5.6,
 0.9,
 0.5,
 2.1,
 4.4,
 2.1,
 2.5,
 7.9,
 3.3,
 -2.7,
 6.0,
 2.1,
 5.6,
 0.9,
 4.0,
 2.1,
 4.4,
 10.3,
 8.3,
 3.2,
 6.0,
 1.7,
 8.0,
 0.9,
 0.5,
 4.0,
 2.9,
 3.2,
 2.8,
 4.8,
 0.1,
 -0.4,
 5.2,
 0.9,
 3.2,
 3.6,
 6.0,
 2.5,
 -0.4,
 3.6,
 4.0,
 2.4,
 -1.2,
 -1.5,
 3.6,
 4.0,
 9.9,
 2.8,
 3.6,
 0.9,
 3.3,
 3.2,
 2.1,
 1.7,
 3.2,
 4.8,
 -1.2,
 0.5,
 6.0,
 0.1,
 4.8,
 3.2,
 7.1,
 -0.4,
 2.1,
 2.1,
 2.4,
 6.4,
 0.5,
 -0.4,
 4.8,
 -2.7,
 3.6,
 2.1,
 2.9,
 5.2,
 4.4,
 0.9,
 -1.9,
 0.5,
 2.1,
 -0.4,
 -3.5,
 0.9,
 2.1,
 5.6,
 3.6,
 1.7,
 -1.2,
 1.7,
 3.2,
 3.6,
 3.2,
 5.6,
 -2.7,
 5.6,


In [29]:
list(map(textstat.flesch_reading_ease, rev_soph_tag_np))

[116.15,
 98.89,
 104.98,
 119.19,
 109.04,
 61.33,
 111.07,
 87.05,
 106.0,
 109.04,
 107.01,
 100.92,
 116.15,
 96.86,
 99.91,
 103.97,
 97.54,
 99.91,
 103.63,
 109.04,
 97.54,
 109.04,
 109.04,
 112.09,
 98.89,
 115.13,
 114.12,
 108.03,
 97.88,
 113.1,
 109.04,
 88.06,
 97.54,
 107.01,
 110.06,
 104.98,
 93.48,
 110.06,
 85.69,
 94.15,
 87.05,
 93.82,
 79.6,
 93.14,
 99.91,
 48.47,
 99.91,
 113.1,
 111.07,
 116.15,
 103.63,
 93.48,
 116.15,
 113.1,
 87.72,
 111.07,
 113.1,
 104.98,
 98.55,
 84.68,
 117.16,
 89.08,
 100.92,
 111.07,
 111.07,
 108.03,
 97.88,
 104.64,
 111.07,
 101.6,
 100.92,
 107.01,
 89.75,
 80.96,
 93.14,
 119.19,
 86.03,
 107.01,
 97.88,
 110.06,
 101.94,
 101.6,
 95.51,
 85.7,
 79.94,
 98.55,
 91.45,
 108.03,
 64.71,
 110.06,
 111.07,
 101.94,
 99.57,
 98.55,
 104.98,
 94.49,
 112.09,
 113.1,
 98.89,
 110.06,
 98.55,
 102.95,
 80.62,
 100.58,
 113.1,
 97.54,
 101.94,
 106.0,
 115.13,
 116.15,
 102.95,
 101.94,
 81.3,
 104.98,
 102.95,
 104.64,
 93.14,
 103.97,

# Reference Code from Papers

### Transformer Paper

In [40]:
# Initialize the NLTK model
def nltk_bleu(texts_origin, text_transfered):
        texts_origin = [word_tokenize(text_origin.lower().strip()) for text_origin in texts_origin]
        text_transfered = word_tokenize(text_transfered.lower().strip())
        return sentence_bleu(texts_origin, text_transfered) * 100

# Check the BLEU diff between original & transferred text
def self_bleu_b(self, texts_origin, texts_transfered):
        assert len(texts_origin) == len(texts_transfered), 'Size of inputs does not match!'
        sum = 0
        n = len(texts_origin)
        for x, y in zip(texts_origin, texts_transfered):
            sum += self.nltk_bleu([x], y)
        return sum / n

### ARAE

In [ ]:

def train_ngram_lm(kenlm_path, data_path, output_path, N):
    """
    Trains a modified Kneser-Ney n-gram KenLM from a text file.
    Creates a .arpa file to store n-grams.
    """
    # create .arpa file of n-grams
    curdir = os.path.abspath(os.path.curdir)
    #
    command = "bin/lmplz -o "+str(N)+" <"+os.path.join(curdir, data_path) + \
              " >"+os.path.join(curdir, output_path)
    os.system("cd "+os.path.join(kenlm_path, 'build')+" && "+command)

    load_kenlm()
    # create language model
    assert(output_path)  # captured by try..except block outside
    model = kenlm.Model(output_path)

    return model


def get_ppl(lm, sentences):
    """
    Assume sentences is a list of strings (space delimited sentences)
    """
    total_nll = 0
    total_wc = 0
    for sent in sentences:
        words = sent.strip().split()
        nll = np.sum([- math.log(math.pow(10.0, score)) for score, _, _ in lm.full_scores(sent, bos=True, eos=False)])
        word_count = len(words)
        total_wc += word_count
        total_nll += nll
    ppl = np.exp(total_nll / total_wc)
    return ppl


### Other - https://github.com/adsieg/text_similarity/blob/master/Different%20Embeddings%20%2B%20Cosine%20Similarity%20%2B%20HeatMap%20illustration.ipynb

In [3]:
def cosine_distance_countvectorizer_method(s1, s2):
    
    # sentences to list
    allsentences = [s1 , s2]
    
    # packages
    from sklearn.feature_extraction.text import CountVectorizer
    from scipy.spatial import distance
    
    # text to vector
    vectorizer = CountVectorizer()
    all_sentences_to_vector = vectorizer.fit_transform(allsentences)
    text_to_vector_v1 = all_sentences_to_vector.toarray()[0].tolist()
    text_to_vector_v2 = all_sentences_to_vector.toarray()[1].tolist()
    
    # distance of similarity
    cosine = distance.cosine(text_to_vector_v1, text_to_vector_v2)
    print('Similarity of two sentences are equal to ',round((1-cosine)*100,2),'%')
    return cosine

In [4]:


def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    with open(gloveFile, encoding="utf8" ) as f:
        content = f.readlines()
    model = {}
    for line in content:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model


def preprocess(raw_text):

    # keep only words
    letters_only_text = re.sub("[^a-zA-Z]", " ", raw_text)

    # convert to lower case and split 
    words = letters_only_text.lower().split()

    # remove stopwords
    stopword_set = set(stopwords.words("english"))
    cleaned_words = list(set([w for w in words if w not in stopword_set]))

    return cleaned_words

def cosine_distance_between_two_words(word1, word2):
    import scipy
    return (1- scipy.spatial.distance.cosine(model[word1], model[word2]))

def calculate_heat_matrix_for_two_sentences(s1,s2):
    s1 = preprocess(s1)
    s2 = preprocess(s2)
    result_list = [[cosine_distance_between_two_words(word1, word2) for word2 in s2] for word1 in s1]
    result_df = pd.DataFrame(result_list)
    result_df.columns = s2
    result_df.index = s1
    return result_df

def cosine_distance_wordembedding_method(s1, s2):
    import scipy
    vector_1 = np.mean([model[word] for word in preprocess(s1)],axis=0)
    vector_2 = np.mean([model[word] for word in preprocess(s2)],axis=0)
    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    print('Word Embedding method with a cosine distance asses that our two sentences are similar to',round((1-cosine)*100,2),'%')

def heat_map_matrix_between_two_sentences(s1,s2):
    df = calculate_heat_matrix_for_two_sentences(s1,s2)
    import seaborn as sns
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=(5,5)) 
    ax_blue = sns.heatmap(df, cmap="YlGnBu")
    # ax_red = sns.heatmap(df)
    print(cosine_distance_wordembedding_method(s1, s2))
    return ax_blue

In [13]:
heat_map_matrix_between_two_sentences(gold[100],raw0[100])

Word Embedding method with a cosine distance asses that our two sentences are similar to 98.39 %
None
